In [2]:
import numpy as np
from pyedflib import highlevel


Script to convert text files with embeddings to EDF files.

In [15]:
with open("test_embeds.txt", "r") as f:
    epochs = np.array([[float(n) for n in x.strip().split()] for x in f.readlines()]).T
epochs /= 10
epochs.shape

(24, 88)

In [16]:
channel_names = [f'ch{x}' for x in range(24)]
# sample freq should be 0.1 for 1 every 10 seconds, this will be 10x (TODO: fix on frontend?)
signal_headers = highlevel.make_signal_headers(channel_names, sample_frequency=0.1) 
#header = highlevel.make_header(patientname='patient_x', gender='Female')
header = highlevel.make_header()
highlevel.write_edf('embeds.edf', epochs, signal_headers, header)

/Users/shy/Projects/CSCI5609/Flower2/env/lib/python3.10/site-packages/pyedflib/edfwriter.py:844: UserWarning: signals are in Fortran order. Will automatically transfer to C order for compatibility with edflib.
  warnings.warn('signals are in Fortran order. Will automatically '


True

In [3]:
# list files in embeds dir with python
import os

for path in os.listdir('embeds'):
    if path == "test_embeds.txt":
        continue
    patient, _, _, session = path.split('.')[0].split('_')
    patient, session = int(patient[1:]), int(session)
    with open(f'embeds/{path}', 'r') as f:
        epochs = np.array([[float(n) for n in x.strip().split()] for x in f.readlines()]).T
    epochs /= 50
    channel_names = [f'ch{x}' for x in range(24)]
    signal_headers = highlevel.make_signal_headers(channel_names, sample_frequency=0.1) 
    #header = highlevel.make_header(patientname='patient_x', gender='Female')
    header = highlevel.make_header()
    highlevel.write_edf(f'edfs/patient{patient:02}_session{session:02}.edf', epochs, signal_headers, header)


/Users/shy/Projects/CSCI5609/Flower2/env/lib/python3.10/site-packages/pyedflib/edfwriter.py:844: UserWarning: signals are in Fortran order. Will automatically transfer to C order for compatibility with edflib.
  warnings.warn('signals are in Fortran order. Will automatically '


In [9]:
%%script true

# read an edf file
signals, signal_headers, header = highlevel.read_edf('edf_file.edf', ch_names=['ch1', 'ch2'])
print(signal_headers[0]['sample_frequency']) # prints 256

# drop a channel from the file or anonymize edf
highlevel.drop_channels('edf_file.edf', to_drop=['ch2', 'ch4'])
highlevel.anonymize_edf('edf_file.edf', new_file='anonymized.edf'
                         to_remove=['patientname', 'birthdate'],
                         new_values=['anonymized', ''])
# check if the two files have the same content
highlevel.compare_edf('edf_file.edf', 'anonymized.edf')
# change polarity of certain channels
highlevel.change_polarity('file.edf', channels=[1,3])
# rename channels within a file
highlevel.rename_channels('file.edf', mapping={'C3-M1':'C3'})